In [9]:
import pandas as pd
import numpy as np
import os
import re

pd.set_option('display.max_rows', 50)

In [10]:
# CALCULATE BUDGETS FOR EACH AREA
# link to source: https://app.clarity.so/rawdao/work/33

month = '2022-04'

init_budget = {}

init_budget['governance'] = 2
init_budget['curation'] = 2.5
init_budget['marketing & pr'] = 2.5
init_budget['community'] = 1.5
init_budget['finance'] = 1.5

perf_tranche = {}

perf_tranche['governance'] =  0.4
perf_tranche['curation'] = 0.4
perf_tranche['marketing & pr'] = 0.4
perf_tranche['community'] = 0.5
perf_tranche['finance'] = 0.4

perf_score = {}

perf_score['governance'] =  1/2
perf_score['curation'] = 3/4
perf_score['marketing & pr'] = 5/5
perf_score['community'] = 1/2
perf_score['finance'] = 2/3


final_budget = {}
total_final_budget = 0
total_init = 0

for area in init_budget:
  final_budget[area] = round(init_budget[area]*(1-perf_tranche[area]) + init_budget[area]*(perf_tranche[area]*perf_score[area]),2)
  total_final_budget = total_final_budget + final_budget[area]
  total_init = total_init + init_budget[area]

print(final_budget)
print("total final budget = " + str(round(total_final_budget,2)) + " out of " + str(total_init))

{'governance': 1.6, 'curation': 2.25, 'marketing & pr': 2.5, 'community': 1.12, 'finance': 1.3}
total final budget = 8.77 out of 10.0


In [11]:
# READ AND CLEAN COORDINAPE DATA
dfs = {}
voting_members = {}
total_give = {}

for file in os.listdir('./coordinape_data'):
  try:
    # read data from coordinape files  
    key = re.search('DAO-(.+?)-', file).group(1).lower()
    # print(key)
    dfs[key] = pd.read_csv('./coordinape_data/'+ file)
    
    # calculate voting stats for each area
    voting_members[key] = int(dfs[key]['sent'].astype(bool).sum(axis=0))
    total_give[key] = int(dfs[key]['received'].sum())

    # calculate compensation per area
    dfs[key] = dfs[key].sort_values(by=['received'], ascending = False) # sort
    dfs[key] = dfs[key][['name','address','received']] # select relevant cols
    dfs[key]['percent_comp'] = dfs[key]['received']/total_give[key] # calculate percentage comp.
    dfs[key]['eth_compensation'] = dfs[key]['percent_comp'] * final_budget[key] # calculate actual ETH comp.
    dfs[key] = dfs[key][dfs[key]['eth_compensation'] > 0] # filter people with 0 compensation
    dfs[key]['area'] = key
    dfs[key]['total_budget_of_area'] = final_budget[key]
  except NameError:
    print(file + ' -> file skipped')
    print(key)
    print(NameError)

# prepare area breakdown for reporting
df_export = pd.concat(dfs)

# merge everything and calculate compensation per member
comp_df = pd.concat(dfs)
comp_df = comp_df[['name','address','eth_compensation']]
comp_df = comp_df.groupby(by=['name','address']).sum().reset_index()
comp_df = comp_df.sort_values(by=['eth_compensation'], ascending = False)

print('# of members who voted per area: ' + str(voting_members))
print('total votes per area: ' + str(total_give))
print("total compensated members: " + str(len(comp_df)))

# of members who voted per area: {'community': 17, 'curation': 16, 'finance': 13, 'governance': 18, 'marketing & pr': 17}
total votes per area: {'community': 1945, 'curation': 1835, 'finance': 1445, 'governance': 1950, 'marketing & pr': 1875}
total compensated members: 37


In [12]:
# PRINT VOTES PER MEMBER PER AREA
df_export[['name','total_budget_of_area','received','percent_comp','eth_compensation']]


name  total_budget_of_area  received  \
community      42           brileigh.eth                  1.12       249   
               25           claudia#0273                  1.12       216   
               15  ezra | shibboleth.eth                  1.12       200   
               41            JoseCactuss                  1.12       197   
               36      matthewbrooks.eth                  1.12       172   
...                                  ...                   ...       ...   
marketing & pr 41         darkroom.felix                  2.50         4   
               13                nyquist                  2.50         3   
               6   alexcartagenamex#4215                  2.50         3   
               30     chadwicktyler#3331                  2.50         3   
               32          foto.eth#8472                  2.50         3   

                   percent_comp  eth_compensation  
community      42      0.128021          0.143383  
               25      0.111054          0.124380  
               15      0.102828          0.115167  
               41      0.101285          0.113440  
               36      0.088432          0.099044  
...                         ...               ...  
marketing & pr 41      0.002133          0.005333  
               13      0.001600          0.004000  
               6       0.001600          0.004000  
               30      0.001600          0.004000  
               32      0.001600          0.004000  

[123 rows x 5 columns]

In [13]:
# PRINT ETH COMPENSATION PER MEMBER FOR THIS MONTH
comp_df[['name','eth_compensation']]

,name,eth_compensation
27,matthewbrooks.eth,1.143952
14,brileigh.eth,1.134215
18,claudia#0273,0.749195
29,munzmeister,0.667239
3,JoseCactuss,0.592920
26,lucaspon,0.535795
35,tatiana#6176,0.468459
4,Lapsus#4632,0.357448
15,bthemouth#0088,0.318344
21,ezra | shibboleth.eth,0.261116


In [16]:
# CALCULATE RANKINGS FOR LONG TERM COMP MODEL

df_rankings = df_export.reset_index()

# This is where we get into a discussion of how exactly to normalize the ranking. The approach I've chosen here is to weigh each member's score by the areas budget, 
# which is equivalent to using their ETH compensation as the score measurement. This way, members who are performing in areas with higher budget (deemed more important) 
# will score higher in the rankings.

df_rankings['normalized_score'] = df_rankings['percent_comp'] * df_rankings['total_budget_of_area']/total_final_budget # score weighted by area's budget
df_rankings = df_rankings[['name','address','normalized_score','area']]
df_rankings = df_rankings.groupby(by=['name','address']).sum()
df_rankings = df_rankings.sort_values(by=['normalized_score'], ascending = False).reset_index()
df_rankings.index = df_rankings.index + 1
df_rankings['percentile'] = df_rankings.index/len(df_rankings)

# Q1-22 Award Conditions
conditions = [
    (df_rankings['percentile'] <= 0.1),
    (df_rankings['percentile'] > 0.1) & (df_rankings['percentile'] <= 0.25),
    (df_rankings['percentile'] > 0.25) & (df_rankings['percentile'] <= 0.5),
    (df_rankings['percentile'] > 0.5) & (df_rankings['percentile'] <= 0.8),
    (df_rankings['percentile'] > 0.8)
]

tier = [ 
    'top 10%',
    'top 25%',
    'top 50%',
    'bottom 50%',
    'bottom 20%'
]

tokens_awarded = [
    0,
    0,
    0,
    0,
    0
]

coordinape = [ 
    20,
    10,
    5,
    0,
    'reset to 100'
]


df_rankings['tier'] = np.select(conditions, tier)
df_rankings['$RAW Awarded'] = np.select(conditions, tokens_awarded)
df_rankings['coordinape_change'] = np.select(conditions, coordinape)


print('total $RAW Awarded = ' + str(df_rankings['$RAW Awarded'].sum()))
df_rankings[['name','normalized_score', 'percentile', 'tier', '$RAW Awarded', 'coordinape_change']]


total $RAW Awarded = 0


,name,normalized_score,percentile,tier,$RAW Awarded,coordinape_change
1,matthewbrooks.eth,0.130439,0.027027,top 10%,0,20
2,brileigh.eth,0.129329,0.054054,top 10%,0,20
3,claudia#0273,0.085427,0.081081,top 10%,0,20
4,munzmeister,0.076082,0.108108,top 25%,0,10
5,JoseCactuss,0.067608,0.135135,top 25%,0,10
6,lucaspon,0.061094,0.162162,top 25%,0,10
7,tatiana#6176,0.053416,0.189189,top 25%,0,10
8,Lapsus#4632,0.040758,0.216216,top 25%,0,10
9,bthemouth#0088,0.036299,0.243243,top 25%,0,10
10,ezra | shibboleth.eth,0.029774,0.270270,top 50%,0,5


In [15]:
# EXPORT RESULTS TO EXCEL
from pathlib import Path
Path("./outputs").mkdir(parents=True, exist_ok=True)

df_export.to_excel("./outputs/"+ month +" - compensation_per_area.xlsx")
comp_df.to_excel("./outputs/"+ month +" - compensation_by_member.xlsx")
df_rankings.to_excel("./outputs/"+ month +" - LT_comp_rankings.xlsx")